In [25]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from math import sqrt
from sklearn.metrics import mean_squared_error

## Q1. Downloading the data

In [26]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

In [27]:
jan_data = pd.read_parquet("../week-1/data/yellow_tripdata_2022-01.parquet")
feb_data = pd.read_parquet("../week-1/data/yellow_tripdata_2022-02.parquet")

In [28]:
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [29]:
print(f'Jan Data Shape {jan_data.shape}')
print(f'Feb Data Shape {feb_data.shape}')

Jan Data Shape (2463931, 19)
Feb Data Shape (2979431, 19)


## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [30]:
jan_data['duration'] = (jan_data.tpep_dropoff_datetime - jan_data.tpep_pickup_datetime)
jan_data['duration'] = jan_data.duration.apply(lambda x: x.total_seconds()/60)

In [31]:
print(jan_data['duration'].std())

46.44530513776499


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [32]:
print(f"{jan_data[~((jan_data.duration >= 1) & (jan_data.duration <=60))].shape[0]} rows was dropped")
jan_data = jan_data[((jan_data.duration >= 1) & (jan_data.duration <=60))]

42491 rows was dropped


98%%

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries
- Fit a dictionary vectorizer
- Get a feature matrix from it
- What's the dimensionality of this matrix (number of columns)?



In [33]:
categorical = ['PULocationID', 'DOLocationID']
jan_data[categorical] = jan_data[categorical].astype(str)

In [34]:
train_dicts = jan_data[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.get_shape()[-1]

515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data
- What's the RMSE on train?

In [41]:
target = 'duration'
y_train = jan_data[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)


In [45]:
rms = sqrt(mean_squared_error(y_train, y_pred, squared=False))
print(f'Root Mean Squared Error {rms}')

Root Mean Squared Error 6.986191065500608


## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

In [36]:
feb_data['duration'] = (feb_data.tpep_dropoff_datetime - feb_data.tpep_pickup_datetime)
feb_data['duration'] = feb_data.duration.apply(lambda x: x.total_seconds()/60)
df_feb = feb_data[((feb_data.duration >= 1) & (feb_data.duration <=60))]
df_feb.PULocationID = df_feb.PULocationID.fillna("-1")
df_feb.DOLocationID = df_feb.DOLocationID.fillna("-1")
categorical = ['PULocationID', 'DOLocationID']
df_feb[categorical] = df_feb[categorical].astype(str)
test_dicts = df_feb[categorical].to_dict(orient='records')
X_test = dv.transform(test_dicts)
y_test = df_feb[target].values


/var/folders/vh/wd57l8mx4zz21bd_2y3pk_t80000gn/T/ipykernel_13372/906587343.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb.PULocationID = df_feb.PULocationID.fillna("-1")
/var/folders/vh/wd57l8mx4zz21bd_2y3pk_t80000gn/T/ipykernel_13372/906587343.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb.DOLocationID = df_feb.DOLocationID.fillna("-1")
/var/folders/vh/wd57l8mx4zz21bd_2y3pk_t80000gn/T/ipykernel_13372/906587343.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a

In [40]:
rms = sqrt(mean_squared_error(y_test, lr.predict(X_test)))
print(f'Root Mean Squared Error {rms}')

Root Mean Squared Error 7.786408015215065
